1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
def convert(numb):
    b = bin(numb)
    h = hex(numb)
    d = int(b,2)
    return print('binary:',b,'\nhexadecimal:',h,'\ndecimal:',d)

convert(75)

binary: 0b1001011 
hexadecimal: 0x4b 
decimal: 75


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
def bitfloat(word):
    m = 1
    list = [p for p in word]
    stri = [str(i) for i in list[1:9]]
    st = "".join(stri)
    e = int('0b'+ st,2)
    for r in range(1,24):
        if list[r+8]=='1' : m += 2**-r
    if list[0] == '0': return print('+', m, 'x 2^(',e-127,')')
    else: return print('-', m, 'x 2^(', e-127, ')')

bitfloat('00000001111100000000000000000000')

+ 1.875 x 2^( -124 )


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
import numpy as np
import sys
up,do,i,j = 1,1,0,0

while up*2. != float('inf'):
    up *= 2.
    i +=1
while do/2. != 0:
    do /= 2.
    j +=1
    
print('Overflow:', up,i,'\nUnderflow:',do,j)

Overflow: 8.98846567431158e+307 1023 
Underflow: 5e-324 1074


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
z,i=1,1
while z!=2 :
    i*=(10**-1)
    z = 2 + i
    print(z)
print('the machine precision is:', format(i*10, '.20f'))

2.1
2.01
2.001
2.0001
2.00001
2.000001
2.0000001
2.00000001
2.000000001
2.0000000001
2.00000000001
2.000000000001
2.0000000000001
2.00000000000001
2.000000000000001
2.0
the machine precision is: 0.00000000000000100000


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [5]:
def solve1(a,b,c):
    x1,x2 = (-b + (b**2-4*a*c)**(1/2))/(2*a), (-b - (b**2-4*a*c)**(1/2))/(2*a)
    return print(x1,x2)
solve1(0.001,1000,0.001)

def solve2(a,b,c):
    x1,x2 = ((-b + (b**2-4*a*c)**(1/2))*(-b - (b**2-4*a*c)**(1/2)))/((2*a)*(-b - (b**2-4*a*c)**(1/2))), ((-b - (b**2-4*a*c)**(1/2))*(-b + (b**2-4*a*c)**(1/2)))/((2*a)*(-b + (b**2-4*a*c)**(1/2)))
    return print(x1,x2)
solve2(0.001,1000,0.001)

#optimized method
def solve3(a,b,c):
    x1,x2 = (-b + (b**2-4*a*c)**(1/2))/(2*a), (-b - (b**2-4*a*c)**(1/2))/(2*a)
    return print(format(x1,'.22f'),format(x2,'.9f'))
solve3(0.001,1000,0.001)

-9.999894245993346e-07 -999999.999999
-9.999894245993346e-07 -999999.9999990001
-0.0000009999894245993346 -999999.999999000


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
def f(x):
    return x*(x-1)
def df(x,d):
    return (f(x+d)-f(x))/(d)

for i in [10**(-2),10**(-4),10**(-6),10**(-8),10**(-10),10**(-12),10**(-14)]: print(df(1,i))

1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
import math as m
import time

def y(x):
    return m.sqrt(1-x**2)
def I(N):
    s = 0
    for k in range(1,N+1): s += y(k*1/N)
    return s*(2/N)

print(I(100))

#computation more or less a second N = 1 100 000
start_time = time.time()
print(I(1100000))
print("--- %s seconds ---" % (time.time() - start_time))

#computation running for one minute (more or less)
start_time = time.time()
print(I(65000000))
print("--- %s seconds ---" % (time.time() - start_time))

1.5602085158895225
1.57079541719438
--- 0.6870565414428711 seconds ---
1.570796311409235
--- 41.42505931854248 seconds ---
